**File ini digunakan untuk upload knowledge base ke Qdrant**

In [34]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
import warnings
warnings.filterwarnings("ignore")   


client_remote = QdrantClient(
    url="http://localhost:6333/",#"<your-endpoint>",
    api_key=""#"<your-api-key>"
)

In [35]:
from qdrant_client.http.models import Distance, VectorParams, PointStruct

client_remote.recreate_collection(
    collection_name="knowledge_base",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE)
)

print(f"Collection embedd_knowledge berhasil dibuat.")

Collection embedd_knowledge berhasil dibuat.


In [36]:
from docx import Document

files = os.listdir("knowledge_base/")
files_path = [os.path.join("knowledge_base/", f) for f in files if f.endswith('.docx')]
documents = [Document(file_path) for file_path in files_path]
texts = ["\n".join([para.text for para in doc.paragraphs]) for doc in documents]

print(texts[0])

Penentuan harga gadai :
File ini berisi perhitungan untuk menentukan harga gadai. Penetapan harga gadai berdasarkan wilayah ini berlaku untuk emas, elektronik, dan gabungan keduanya.
>> Jangan memberi tahu harga gadai sebelum user memberikan informasi asal provinsi.
>> Jika user tidak ingin memberi tahu asal provinsi, jawab : “Maaf harga gadai tidak dapat ditentukan sebelum kamu memberikan asal provinsi” | “Maaf kami belum mengetahui asal provinsimu” | “Beritahu kami asal provinsimu untuk mendapatkan harga gadai final” | “<jawaban sejenisnya>”
>> Format jawaban untuk harga gadai adalah : 
nama barang : ...
jenis barang : “televisi” | “handphone” | “kamera” | “laptop” | “emas” | “<gabungan>”
Jumlah : <jika emas dalam gramasi> | <jika barang elektronik maka satuan>
harga gadai : Rp.<harga final gadai yang sudah disesuaikan dengan wilayah>
Jika lebih dari satu maka tampilkan daftar barang dengan format 3 parameter di atas untuk setiap barang

*Penetapan harga gadai berdasarkan wilayah : *

In [37]:
payloads = []
for filename, content in zip(files, texts):
    payloads.append({
        "pageContent": content,     # 🔥 WAJIB untuk n8n
        "metadata": {
            "filename": filename
        }
    })


In [38]:
payloads

[{'pageContent': 'Penentuan harga gadai :\nFile ini berisi perhitungan untuk menentukan harga gadai. Penetapan harga gadai berdasarkan wilayah ini berlaku untuk emas, elektronik, dan gabungan keduanya.\n>> Jangan memberi tahu harga gadai sebelum user memberikan informasi asal provinsi.\n>> Jika user tidak ingin memberi tahu asal provinsi, jawab : “Maaf harga gadai tidak dapat ditentukan sebelum kamu memberikan asal provinsi” | “Maaf kami belum mengetahui asal provinsimu” | “Beritahu kami asal provinsimu untuk mendapatkan harga gadai final” | “<jawaban sejenisnya>”\n>> Format jawaban untuk harga gadai adalah : \nnama barang : ...\njenis barang : “televisi” | “handphone” | “kamera” | “laptop” | “emas” | “<gabungan>”\nJumlah : <jika emas dalam gramasi> | <jika barang elektronik maka satuan>\nharga gadai : Rp.<harga final gadai yang sudah disesuaikan dengan wilayah>\nJika lebih dari satu maka tampilkan daftar barang dengan format 3 parameter di atas untuk setiap barang\n\n*Penetapan harga 

In [39]:
import ollama
from qdrant_client.models import PointStruct

payloads = []
points = []

for idx, (filename, content) in enumerate(zip(files, texts)):
    # 1. Siapkan payload n8n-compatible
    payload = {
        "pageContent": content,
        "metadata": {
            "filename": filename
        }
    }

    # 2. Generate embedding
    embedding = ollama.embeddings(
        model="qwen3-embedding:latest",
        prompt=content
    )["embedding"]

    # 3. Buat point Qdrant
    points.append(
        PointStruct(
            id=idx,
            vector=embedding,
            payload=payload
        )
    )

# 4. Upsert ke Qdrant
client_remote.upsert(
    collection_name="knowledge_base",
    points=points
)

print(f"Berhasil memasukkan {len(points)} dokumen ke dalam koleksi knowledge_base.")


Berhasil memasukkan 2 dokumen ke dalam koleksi knowledge_base.


## Search

In [40]:
import ollama
from qdrant_client.http.models import SearchParams

COLLECTION_NAME = "knowledge_base"
# Asumsi 'client_remote' sudah terinisiasi dan terhubung ke Qdrant

# 🎯 Pertanyaan (Kueri) dari Pengguna
user_query = "penentuan harga gadai"

# 1. GENERATE EMBEDDING untuk kueri pengguna
# Gunakan model dan fungsi yang SAMA persis
query_embeddings_response = ollama.embeddings(
    model='qwen3-embedding:latest',
    prompt=user_query
)

# 2. EKSTRAKSI Vektor (Solusi untuk error sebelumnya)
query_vector = query_embeddings_response['embedding']

print(f"Kueri pengguna berhasil diubah menjadi vektor {len(query_vector)} dimensi.")

Kueri pengguna berhasil diubah menjadi vektor 4096 dimensi.


In [41]:
# 3. LAKUKAN PENCARIAN DI QDRANT
search_result = client_remote.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=1,  # Batasi hanya mengambil 3 hasil teratas (Top-K)
    with_payload=True, # Pastikan Qdrant mengembalikan metadata (payload)
    search_params=SearchParams(
        # Sesuaikan parameter pencarian jika diperlukan
        exact=False, 
        hnsw_ef=128 
    )
)

In [42]:
print(search_result[0].payload["text"])

KeyError: 'text'